![Proxy](Proxy.png)

1. A Interface __Object__ declara a interface do Serviço. O proxy deve seguir essa interface para ser capaz de se disfarçar como um objeto do serviço

2. O __RealObject__ é uma classe que fornece alguma lógica de negócio útil.

3. A classe __Proxy__ tem um campo de referência que aponta para um objeto do serviço. Após o proxy finalizar seu processamento (por exemplo: inicialização preguiçosa, acesso, acessar controle, colocar em cache, etc.), ele passa o pedido para o objeto do serviço.

   Geralmente os proxies gerenciam todo o ciclo de vida dos seus objetos de serviço.

4. O __Client__ deve trabalhar tanto com os serviços e proxies através da mesma interface. Dessa forma você pode passar uma proxy para qualquer código que espera um objeto do serviço.

In [16]:
from __future__ import annotations
from typing import List, Dict
from abc import ABCMeta, abstractmethod

In [17]:
class IAccessData(metaclass = ABCMeta):
    """ Interface: Object """
    def __init__(self, user, password) -> None: pass
    
    @abstractmethod
    def login(self) -> None: pass

    @abstractmethod
    def getData(self) -> None: pass

    @abstractmethod
    def logout(self) -> None: pass

In [18]:
class AccessData(IAccessData):
    """ RealObject """

    def __init__(self, user: str, password: str) -> None:
        self.__user = user
        self.__password = password
        self.__session = False

        self.account = {'user': 'deroncio', 'password': '12345'}

    @property
    def session(self):
        return self.__session

    def login(self) -> bool:
        if self.__user == self.account['user'] and self.__password == self.account['password']:
            self.__session = True
            return True
        print('Unauthorized access! Please your credentials.')
        return False
    
    def getData(self) -> List[Dict]:
        if self.__session:
            return [{'success': True, 'addresses': {'logradouro':'Av. Brasil', 'num':1010}}]
        print('Please login!')
        return [{'success': False}]
    
    def logout(self) -> str:
        if self.__session:
            self.__session = False
            print('[success] logout successfully')
        else:
            print('[fail] Please login!')
        return self.__session

In [19]:
class ProxyAccessData(IAccessData):
    """ RealObject """

    def __init__(self, user: str, password: str) -> None:
        self.__user = user
        self.__password = password

        self.__access: AccessData
        self.__cache: List[Dict]

        if not hasattr(self, '__access'):
            self.__access = AccessData(self.__user, self.__password)

    def login(self) -> None:
        if not hasattr(self, '__session'):
            self.__access.login()

    def getData(self) -> None:
        if not hasattr(self, '__cache'):
            self.__cache = self.__access.getData()
            return self.__cache

    def logout(self) -> None:
        self.__access.logout()

In [21]:
if __name__ == "__main__":

    user_proxy = ProxyAccessData(user='deroncio', password='12345')

    if user_proxy.login():
        data = user_proxy.getData()

        print(data)

[{'success': True, 'addresses': {'logradouro': 'Av. Brasil', 'num': 1010}}]
